分步说明：

1. 用户输入自然语言查询
2. 系统先去检索跟查询相关的表
3. 根据表的 Schema 让大模型生成 SQL
4. 用生成的 SQL 查询数据库
5. 根据查询结果，调用大模型生成自然语言回复

1. 数据遍历，当前的目录下

In [2]:
import pandas as pd
from pathlib import Path

data_dir = Path("./WikiTableQuestions/csv/200-csv")
csv_files = sorted([f for f in data_dir.glob("*.csv")])
dfs = []
for csv_file in csv_files:
    print(f"processing file: {csv_file}")
    try:
        df = pd.read_csv(csv_file)
        dfs.append(df)
    except Exception as e:
        print(f"Error parsing {csv_file}: {str(e)}")

processing file: WikiTableQuestions\csv\200-csv\0.csv
processing file: WikiTableQuestions\csv\200-csv\1.csv
processing file: WikiTableQuestions\csv\200-csv\10.csv
processing file: WikiTableQuestions\csv\200-csv\11.csv
processing file: WikiTableQuestions\csv\200-csv\12.csv
processing file: WikiTableQuestions\csv\200-csv\14.csv
processing file: WikiTableQuestions\csv\200-csv\15.csv
Error parsing WikiTableQuestions\csv\200-csv\15.csv: Error tokenizing data. C error: Expected 4 fields in line 16, saw 5

processing file: WikiTableQuestions\csv\200-csv\17.csv
Error parsing WikiTableQuestions\csv\200-csv\17.csv: Error tokenizing data. C error: Expected 6 fields in line 5, saw 7

processing file: WikiTableQuestions\csv\200-csv\18.csv
processing file: WikiTableQuestions\csv\200-csv\20.csv
processing file: WikiTableQuestions\csv\200-csv\22.csv
processing file: WikiTableQuestions\csv\200-csv\24.csv
processing file: WikiTableQuestions\csv\200-csv\25.csv
processing file: WikiTableQuestions\csv\200-

2. 为每个表生成一段文字表述（用于检索），保存在 `WikiTableQuestions_TableInfo` 目录

In [3]:
from llama_index.core.prompts import ChatPromptTemplate
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.llms import ChatMessage


class TableInfo(BaseModel):
    """Information regarding a structured table."""

    table_name: str = Field(
        ..., description="table name (must be underscores and NO spaces)"
    )
    table_summary: str = Field(
        ..., description="short, concise summary/caption of the table"
    )


prompt_str = """
Give me a summary of the table with the following JSON format.

- The table name must be unique to the table and describe it while being concise.
- Do NOT output a generic table name (e.g. table, my_table).

Do NOT make the table name one of the following: {exclude_table_name_list}

Table:
{table_str}

Summary: """

prompt_tmpl = ChatPromptTemplate(
    message_templates=[ChatMessage.from_str(prompt_str, role="user")]
)

In [4]:
tableinfo_dir = "WikiTableQuestions_TableInfo"

In [ ]:
import json


def _get_tableinfo_with_index(idx: int) -> str:
    results_gen = Path(tableinfo_dir).glob(f"{idx}_*")
    results_list = list(results_gen)
    if len(results_list) == 0:
        return None
    elif len(results_list) == 1:
        path = results_list[0]
        with open(path, 'r') as file:
            data = json.load(file)
            return TableInfo.model_validate(data)
    else:
        raise ValueError(
            f"More than one file matching index: {list(results_gen)}"
        )


table_names = set()
table_infos = []
for idx, df in enumerate(dfs):
    table_info = _get_tableinfo_with_index(idx)
    if table_info:
        table_infos.append(table_info)
    else:
        while True:
            df_str = df.head(10).to_csv()
            table_info = llm.structured_predict(
                TableInfo,
                prompt_tmpl,
                table_str=df_str,
                exclude_table_name_list=str(list(table_names)),
            )
            table_name = table_info.table_name
            print(f"Processed table: {table_name}")
            if table_name not in table_names:
                table_names.add(table_name)
                break
            else:
                # try again
                print(f"Table name {table_name} already exists, trying again.")
                pass

        out_file = f"{tableinfo_dir}/{idx}_{table_name}.json"
        json.dump(table_info.dict(), open(out_file, "w"))
    table_infos.append(table_info)